In [ ]:
# Example: Assuming your LSTM + CNN model is defined as MyModel
model = MyModel()
criterion = nn.MSELoss(reduction='none')  # Use 'none' to get individual losses

# Compute or define your grid area weights
grid_weights = torch.tensor([[weight_lat_lon for lon in range(24)] for lat in range(50)])  # Example weights

optimizer = optim.Adam(model.parameters(), lr=0.001)

def weighted_loss(output, target, weights):
    loss = criterion(output, target)  # Shape: (batch_size, 50, 24)
    weighted_loss = loss * weights  # Broadcasting weights over the batch dimension
    return weighted_loss.mean()

# Training loop
for epoch in range(num_epochs):
    for inputs, targets in data_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        
        # Assuming outputs and targets are of shape (batch_size, 50, 24)
        weights = grid_weights.to(outputs.device)  # Move weights to the same device as outputs
        loss = weighted_loss(outputs, targets, weights)
        
        loss.backward()
        optimizer.step()

In [2]:
import numpy as np

In [5]:
def calculate_delta(lat1, lon1, lat2, lon2):
    R_E = 6371.0 * 1e3 #meters

    lat1_rad = np.radians(lat1)
    lon1_rad = np.radians(lon1)
    lat2_rad = np.radians(lat2)
    lon2_rad = np.radians(lon2)

    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    delta_x = (dlat / 360)*2*np.pi*R_E
    delta_y = (dlon / 360)*2*np.pi*R_E*np.sin(lat1_rad)

    return delta_x, delta_y

lat = np.linspace(53.1, 89.7, 30) #30 intervals between 53.1° to 89.7°
lon = np.linspace(1.6, 357.6, 80) #80 intervals between 1.6° to 357.6°

delta_x, delta_y = calculate_delta(lat[0], lon[0], lat[1], lon[1])